# GeoAPI Client Notebook

This notebook tests a locally hosted **Geo Risk API**.

**Endpoints covered**:
- `/healthz`
- `/risk/targets`
- `/risk/summary`
- `/risk/features/{target}.geojson`
- `/risk/summary/top`
- `/risk/zone.geojson`

You can configure the base URL, dam identifier, and targets in the next cell.

In [1]:
# ---- Configuration ----
import os
BASE_URL   = os.environ.get("GEOAPI_BASE_URL", "http://localhost:8080")
DAMNUMBER  = os.environ.get("GEOAPI_DAMNUMBER", "UT00259")  # set to None to use DAM_NAME
DAM_NAME   = os.environ.get("GEOAPI_DAM_NAME")              # e.g., "Something Dam"
TARGETS    = os.environ.get("GEOAPI_TARGETS", "power_plants,railroads").split(",")

CLIP   = True
LIMIT  = 200
OFFSET = 0

def qbool(b: bool) -> str:
    return "true" if b else "false"

DAM_PARAM = {"damnumber": DAMNUMBER} if DAMNUMBER else {"dam_name": DAM_NAME}
assert any(DAM_PARAM.values()), "Please set DAMNUMBER or DAM_NAME"


In [2]:
# ---- Helper functions ----
import requests, json

def get(path: str, **params):
    url = f"{BASE_URL}{path}"
    r = requests.get(url, params=params, timeout=60)
    if r.status_code >= 400:
        print("URL:", r.url)
        print("Status:", r.status_code)
        try:
            print(r.json())
        except Exception:
            print(r.text[:500])
        r.raise_for_status()
    try:
        return r.json()
    except Exception:
        print("Non-JSON response:", r.text[:500])
        raise

def features_to_df(fc: dict):
    """Flatten a GeoJSON FeatureCollection to a pandas DataFrame of properties."""
    import pandas as pd
    feats = fc.get("features", [])
    rows = []
    for f in feats:
        props = (f.get("properties") or {}).copy()
        geom = f.get("geometry") or {}
        props["_geom_type"] = geom.get("type")
        rows.append(props)
    return pd.DataFrame(rows)

def save_geojson(obj: dict, filename: str):
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(obj, f)
    return os.path.abspath(filename)


## 1) Health check & targets

In [3]:
health = get("/healthz")
health

{'status': 'ok'}

In [4]:
available_targets = get("/risk/targets")["targets"]
available_targets

['aviation',
 'gap_status',
 'hazardous_waste',
 'hospitals',
 'ng_pipelines',
 'power_plants',
 'railroads',
 'svi_tracts',
 'transportation',
 'wwtp']

## 2) Summary counts for selected targets

In [5]:
summary = get(
    "/risk/summary",
    targets=",".join([t.strip() for t in TARGETS if t.strip()]),
    clip="false",  # counts ignore clip; kept for symmetry
    **DAM_PARAM,
)
summary

{'damnumber': 'UT00259',
 'dam_name': 'Rocky Ford (Beaver)',
 'counts': {'power_plants': 2, 'railroads': 6}}

## 3) Fetch features (GeoJSON) for each target

In [6]:
import pandas as pd
dfs = {}
for t in [t.strip() for t in TARGETS if t.strip()]:
    print(f"Fetching target: {t}")
    fc = get(
        f"/risk/features/{t}.geojson",
        clip=qbool(CLIP),
        limit=LIMIT,
        offset=OFFSET,
        **DAM_PARAM,
    )
    if not isinstance(fc, dict) or fc.get("type") != "FeatureCollection":
        print("Unexpected response for target", t, "->", fc)
        continue
    n = len(fc.get("features", []))
    print(f"{t}: {n} features")
    try:
        df = features_to_df(fc)
        dfs[t] = df
        display(df.head())
    except Exception as e:
        print("(Optional) pandas not available or error flattening:", e)
    # Save GeoJSON for inspection
    out = save_geojson(fc, f"{t}.geojson")
    print("Saved:", out)


Fetching target: power_plants
power_plants: 2 features


,zip,city,name,type,lines,state,sub_1,sub_2,county,source,...,plant_code,retir_unit,retire_cap,sourc_long,source_lat,sourcedate,summer_cap,val_method,winter_cap,_geom_type
0,84751,MILFORD,MILFORD FLAT SOLAR PLANT,SOLAR PHOTOVOLTAIC,0,UT,NOT AVAILABLE,NOT AVAILABLE,BEAVER,"ESRI, Google Imagery, and EIA Data",...,58601,0,0,-113.008333,38.291389,2019-04-25T00:00:00+00:00,3,IMAGERY/OTHER,3,Point
1,84751,MILFORD,GRANITE PEAK SOLAR PLANT,SOLAR PHOTOVOLTAIC,0,UT,NOT AVAILABLE,NOT AVAILABLE,BEAVER,"ESRI, Google Imagery, and EIA Data",...,58604,0,0,-112.988889,38.402778,2019-04-25T00:00:00+00:00,3,IMAGERY/OTHER,3,Point


Saved: /Users/junghawoo/Documents/github/geo-risk-largest-full/clients/power_plants.geojson
Fetching target: railroads
railroads: 6 features


,name,suftyp,basename,objectid,length_mi,length_yd,stgeometry,suftypeabr,shape_length,_geom_type
0,Union Pacific RR,505,Union Pacific,18394,0.994209,1749.807831,2046.424951,RR,2046.424951,MultiLineString
1,Union Pacific RR,505,Union Pacific,18395,5.055939,8898.453250,10418.633801,RR,10418.633801,LineString
2,,0,,18402,0.142793,251.315719,293.905150,,293.905150,LineString
3,,0,,18403,0.474632,835.352272,975.843139,,975.843139,LineString
4,Union Pacific RR,505,Union Pacific,109749,22.767633,40071.033538,46732.864545,RR,46732.864545,LineString


Saved: /Users/junghawoo/Documents/github/geo-risk-largest-full/clients/railroads.geojson


## 4) Zone geometry

In [7]:
zone = get("/risk/zone.geojson", **DAM_PARAM)
print("Zone features:", len(zone.get("features", [])))
save_geojson(zone, "zone.geojson")
zone.get("features", [])[:1]  # preview first feature

Zone features: 1


[{'type': 'Feature',
  'geometry': {'type': 'MultiPolygon',
   'coordinates': [[[[-113.031019202, 38.376274382],
      [-113.030403121, 38.376817487],
      [-113.029736012, 38.377148504],
      [-113.029116941, 38.377585169],
      [-113.027700479, 38.378532566],
      [-113.026728743, 38.379259283],
      [-113.025662887, 38.37984564],
      [-113.025172055, 38.380031603],
      [-113.024553921, 38.380503722],
      [-113.023790675, 38.380623398],
      [-113.022856038, 38.381065482],
      [-113.022323082, 38.381358645],
      [-113.021794074, 38.381793718],
      [-113.020995613, 38.382268932],
      [-113.020242214, 38.382743363],
      [-113.019360492, 38.383468468],
      [-113.018478753, 38.384193566],
      [-113.017912515, 38.384913221],
      [-113.017519664, 38.385381422],
      [-113.017037645, 38.385886654],
      [-113.016650695, 38.38656772],
      [-113.016303888, 38.387070621],
      [-113.015865939, 38.387539593],
      [-113.015262454, 38.388543837],
      [-113.014

## 5) Top-N dams by a target

In [8]:
top = get("/risk/summary/top", target="railroads", n=20)
import pandas as pd
pd.DataFrame(top["top"]).head()

,damnumber,dam_name,count
0,UT00580,South Ogden City Burch Creek Debris,36
1,UT00221,Mountain Dell,23
2,UT00505,Ogden City - Sullivan Hollow,20
3,UT10101,Red Butte Dam,19
4,UT00272,Sevier Bridge,9


## 6) Optional: bbox & simplify example
Adjust the `example_bbox` string to your area of interest.

In [9]:
example_bbox = "-112.35,40.61,-112.20,40.73"  # minx,miny,maxx,maxy in EPSG:4326
fc_bbox = get(
    f"/risk/features/{TARGETS[0]}.geojson",
    bbox=example_bbox,
    clip="false",
    limit=50,
    offset=0,
    **DAM_PARAM,
)
len(fc_bbox.get("features", []))

0

In [10]:
# Simplify (meters). Note: done server-side in EPSG:3857 then reprojected to 4326.
fc_simplified = get(
    f"/risk/features/{TARGETS[0]}.geojson",
    simplify_m=50,
    clip=qbool(CLIP),
    limit=LIMIT,
    offset=OFFSET,
    **DAM_PARAM,
)
len(fc_simplified.get("features", []))

2

## 7) (Optional) Quick map with Folium
If you have `folium` installed, this will overlay the zone and a target layer.

In [12]:
import json
import folium
from IPython.display import display

# Ensure these two are Python dicts (FeatureCollections) not raw strings
zone_fc = zone if isinstance(zone, dict) else json.loads(zone)
feats_fc = fc_simplified if isinstance(fc_simplified, dict) else json.loads(fc_simplified)

m = folium.Map(location=[40.7, -111.9], zoom_start=8, control_scale=True)
folium.GeoJson(zone_fc, name="Inundation Zone").add_to(m)
if TARGETS:
    folium.GeoJson(feats_fc, name=TARGETS[0]).add_to(m)
folium.LayerControl().add_to(m)

display(m)  # <- explicit display works in all Jupyter frontends


In [16]:
import os
import requests
import pandas as pd
from IPython.display import display

# ---- Config ----
BASE = os.getenv("GEOAPI_URL", "http://localhost:8080")
ENDPOINT = f"{BASE}/risk/metrics"

# Ask GeoAPI for ALL dams and ALL targets, using the precomputed cache
params = {
    "damnumber": "all",
    "targets": "all",
    "precomputed": "true",
    "length":"mi"
}

r = requests.get(ENDPOINT, params=params, timeout=180)
r.raise_for_status()
payload = r.json()

# Expect: {"items": [ {damnumber, dam_name, <target1>, <target2>, ...}, ... ]}
items = payload.get("items", [])
if not items:
    raise ValueError("No items returned from GeoAPI. Check that the cache is built and the endpoint is reachable.")

df = pd.DataFrame(items)

# Reorder columns: dam identifiers first, then metrics
id_cols = ["damnumber", "dam_name"]
metric_cols = [c for c in df.columns if c not in id_cols]
df = df[id_cols + metric_cols]

# Make numeric where possible (in case some fields come back as strings)
for c in metric_cols:
    df[c] = pd.to_numeric(df[c], errors="ignore")

# Optional pretty formatting:
POINT = {"aviation", "hazardous_waste", "hospitals", "power_plants", "wwtp"}         # counts
LINE  = {"ng_pipelines", "railroads", "transportation"}                               # length (meters)
POLY  = {"gap_status", "svi_tracts"}                                                  # area (m²)

for c in metric_cols:
    if c in POINT:
        # cast to nullable int if numeric
        try:
            df[c] = pd.to_numeric(df[c], errors="coerce").astype("Int64")
        except Exception:
            pass
    elif c in LINE | POLY:
        # round numeric columns for readability
        try:
            df[c] = pd.to_numeric(df[c], errors="coerce").round(1)
        except Exception:
            pass

# Show the full table (Jupyter will paginate if it's large in some UIs)
display(df)

# Save to CSV
out_csv = "geoapi_risk_metrics_all.csv"
df.to_csv(out_csv, index=False)
print(f"Saved {len(df):,} rows to {out_csv}")


,damnumber,dam_name,aviation,gap_status,hazardous_waste,hospitals,ng_pipelines,power_plants,railroads,svi_tracts,transportation,wwtp
0,UT00002,Adams,0,0.0,0,0,0.0,0,0.2,2142832.9,2.7,0
1,UT00010,Ash Creek,0,1830141.7,0,0,0.0,0,0.0,21207545.4,3.9,0
2,UT00024,Box Creek - Lower (Beaver Creek),0,0.0,0,0,0.0,0,0.0,2037377.6,0.0,0
3,UT00025,Box Creek - Upper (Beaver Creek),0,0.0,0,0,0.0,0,0.0,2353544.0,0.0,0
4,UT00034,Big East,0,0.0,0,0,0.0,0,0.0,2997094.9,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
222,UT53257,Oquirrh Lake Dam/Kennecott Daybreak,0,0.0,0,0,0.0,0,0.0,1024152.4,0.0,0
223,UT53277,Riverton City - Black Ridge Reservoir,0,67226.5,0,0,0.8,0,0.0,2496175.4,0.0,0
224,UT53326,Big Sand Wash Dam,1,0.0,0,0,4.2,0,0.0,130023725.1,0.0,0
225,UT53352,M&S Dam,0,0.0,0,0,0.0,0,0.0,9190379.3,0.0,0


Saved 227 rows to geoapi_risk_metrics_all.csv
